# Лабораторная работа №3
### Выполнил студент группы БВТ1905 Нестеров Юрий
## Задание №1

In [2]:
from termcolor2 import c
import ipywidgets as wg
import random
import time


user_string = wg.Textarea(value='', placeholder='', description='Введите строку: ', disabled=False)  # User inputs in notebook
user_substring = wg.Text(value='', placeholder='', description='Введите подстроку: ', disabled=False)
user_letter_case = wg.Checkbox(value=False, description="Учитывать регистр", disabled=False, indent=False)

### Стандартный алгоритм поиска подстроки в языке Python

In [11]:
def standart(sub, string, case):
    let_str = string  # copying original string
    if not case:
        string = string.lower()  # decapitalize all capital letters for ignoring cases
    return string.find(sub)


display(user_string, user_substring, user_letter_case)
time_start = time.monotonic()
print(standart(user_substring.value, user_string.value, user_letter_case.value))
time_stop = time.monotonic() - time_start
print("Time: {0} s".format(time_stop))

Textarea(value='', description='Введите строку: ', placeholder='')

Text(value='', description='Введите подстроку: ', placeholder='')

Checkbox(value=False, description='Учитывать регистр', indent=False)

0
Time: 0.01599999999996271 s


### Алгоритм поиска Кнута-Морриса-Пратта

In [5]:
def reversed(str):  # additional function for line reversal
    return str[::-1]


def pratt(sub, string, case):
    let_str = string  # copying original string
    if not case:
        string = string.lower()  # decapitalize all capital letters for ignoring cases
    while True:
        symbol = chr(random.randint(0, 512))  # finding a random symbol that doesn't use in sub and string
        if symbol not in sub + string:
            break
    full = sub + symbol + string
    iter_array = [0] * len(full)  # making a list with number of overlaping symbols
    for i in range(1, len(full)):
        search, reverse = full[0:i], full[i::-1]
        for j in range(1, len(search)):
            if search[0:j] == reversed(reverse[0:j]):
                iter_array[i] = j
    for i in range(len(iter_array)):  # print to screen, if we find an overlap
        if iter_array[i] == len(sub):
            print(let_str[:i-(len(sub)*2)] + c(let_str[i-(len(sub)*2):i-len(sub)]).on_yellow + let_str[i-len(sub):])


display(user_string, user_substring, user_letter_case)
time_start = time.monotonic()
pratt(user_substring.value, user_string.value, user_letter_case.value)
time_stop = time.monotonic() - time_start
print("Time: {0} s".format(time_stop))

Textarea(value='Hoola-hoola girls like hooligan', description='Введите строку: ', placeholder='')

Text(value='hooligan', description='Введите подстроку: ', placeholder='')

Checkbox(value=False, description='Учитывать регистр', indent=False)

Hoola-hoola girls like hooligan
Time: 0.0 s


### Упрощённый алгоритм Бойера-Мура

In [6]:
def mour(sub, string, case):
    let_str = string
    if not case:
        string = string.lower()
    alphabet_dict = {}
    count = 0; res = []
    for i in sub[::-1]:
        if i not in alphabet_dict:
            alphabet_dict[i] = count
        count += 1
    match = len(sub)-1
    while match < len(string):
        if sub[-1] == string[match]:
            k = True
            for i in range(1, len(sub)):
                if sub[-1-i] != string[match-i]:
                    k = False
                    break
            if k:
                res.append(match-len(sub))
            match += 1
        elif alphabet_dict.get(string[match]) is not None:
            match += alphabet_dict.get(string[match])
        else:
            match += len(sub)
    for i in range(len(res)):
        print(let_str[:res[i]+1] + c(let_str[res[i]+1:res[i]+len(sub)+1]).on_yellow + let_str[res[i]+len(sub)+1:])
        

display(user_string, user_substring, user_letter_case)
if len(user_string.value) > 0 and len(user_substring.value) > 0:
    time_start = time.monotonic()
    mour(user_substring.value, user_string.value, user_letter_case.value)
    time_stop = time.monotonic() - time_start
    print("Time: {0} s".format(time_stop))

Textarea(value='Hoola-hoola girls like hooligan', description='Введите строку: ', placeholder='')

Text(value='hooligan', description='Введите подстроку: ', placeholder='')

Checkbox(value=False, description='Учитывать регистр', indent=False)

Hoola-hoola girls like hooligan
Time: 0.0 s


## Задание №2

In [1]:
start = [
            [ 1,  6,  7,  5],
            [ 9,  3, 10,  2],
            [13,  8,  4, 12],
            [14, 11, 15,  0]
        ]

In [7]:
import copy
from pprint import pprint


class Node:
    def __init__(self, position, moves, heuristic):  # Builds a new node
        self._position = position
        self._moves = moves
        self._heuristic = heuristic
        self._h_score = None

    def get_position(self):  # Get position of the Node
        return copy.deepcopy(self._position)

    def get_g_score(self):  # Get gScore of the Node
        return len(self._moves)

    def get_h_score(self):  # Get hScore of this Node. Heuristic passed in the constructor will be used for computation
        if self._h_score is None:
            self._h_score = self._heuristic.compute(self)
        return self._h_score

    def get_f_score(self):  # Get fScore of the Node fScore = gScore + hScore
        return self.get_g_score() + self.get_h_score()

    def get_moves(self):  # Return list of moves which bring us to this position
        return copy.copy(self._moves)

    def get_heuristic(self):  # Return heuristic used to compute hScore for this node
        return self._heuristic

    def get_coord(self, value):  # Get i and j coord of the given value
        i = 0
        for row in self._position:
            j = 0
            for cell in row:
                if cell == value:
                    return [i, j]
                j += 1
            i += 1



class NodeBuilder:  # Build child nodes for 15-puzzle
    def get_child_nodes(self, node):  # Return list of valid child nodes
        children = []
        i_space, j_space = node.get_coord(0)
        for i in range(-1, 2):
            for j in range(-1, 2):
                if i * j != 0 or i == j:
                    continue
                i_swap, j_swap = i_space + i, j_space + j
                if not (0 <= i_swap <= 3) or not (0 <= j_swap <= 3):
                    continue
                position = node.get_position()
                position[i_space][j_space] = position[i_swap][j_swap]
                position[i_swap][j_swap] = 0
                moves = node.get_moves()
                moves.append(self._get_move_name(i, j))
                child = Node(position, moves, node.get_heuristic())
                children.append(child)
        return children

    def _get_move_name(self, i_delta, j_delta):
        if i_delta == -1:
            return 'up'
        if i_delta == 1:
            return 'down'
        if j_delta == -1:
            return 'left'
        if j_delta == 1:
            return 'right'


class NodePool:
    def __init__(self):
        self._pool = []
        self._history = {}

    def add(self, node):
        if str(node.get_position()) in self._history:
            # duplicate entry
            return
        self._history[str(node.get_position())] = True
        self._insort(node)

    def pop(self):
        return self._pool.pop(0)

    def is_empty(self):
        return len(self._pool) == 0

    def _insort(self, node):
        low = 0
        high = len(self._pool)
        while low < high:
            mid = (low + high) // 2
            if node.get_f_score() < self._pool[mid].get_f_score():
                high = mid
            else:
                low = mid + 1
        self._pool.insert(low, node)


class ManhattanDistance:
    def __init__(self):
        self._goal = Node([
            [ 1,  2,  3,  4],
            [ 5,  6,  7,  8],
            [ 9, 10, 11, 12],
            [13, 14, 15,  0]
        ], [], self)

    def compute(self, node):  # Computes Manhattan distance of the given Node
        score = 0
        for value in range(1, 16):
            i_goal, j_goal = self._goal.get_coord(value)
            i_actual, j_actual = node.get_coord(value)
            score += abs(i_goal - i_actual) + abs(j_goal - j_actual)
        return score


class AStar:
    def __init__(self, heuristic):
        self._node_pool = NodePool()
        self._node_builder = NodeBuilder()
        self._heuristic = heuristic

    def solve(self, position):
        self._bootstrap(position)
        while not self._node_pool.is_empty():
            cur_node = self._node_pool.pop()
            if cur_node.get_h_score() == 0:
                return cur_node.get_moves()
            children = self._node_builder.get_child_nodes(cur_node)
            for child in children:
                self._node_pool.add(child)
        return None

    def _bootstrap(self, position):
        node = Node(position, [], self._heuristic)
        self._node_pool.add(node)


heuristic = ManhattanDistance()
astar = AStar(heuristic)

start_time = time.monotonic()
result = astar.solve(start)

if result is None:
    print('There is no solution')
else:
    pprint(result)
    print('Need: {0} moves'.format(len(result)))
    print('Time: {0} s.'.format((time.monotonic() - start_time)))

['up',
 'left',
 'left',
 'up',
 'right',
 'up',
 'right',
 'down',
 'down',
 'left',
 'up',
 'up',
 'right',
 'down',
 'down',
 'left',
 'left',
 'up',
 'right',
 'down',
 'right',
 'down',
 'left',
 'left',
 'left',
 'up',
 'up',
 'right',
 'up',
 'right',
 'down',
 'down',
 'down',
 'right']
Need: 34 moves
Time: 11.015999999988708 s.


## Вывод

В процессе выполнения данной работы я реализовал такие методы поиска подстроки в строке, как метод Кнута-Морисса-Пратта и упрощённый метод Бойера-Мура. Для каждого из поисков реализован графический вывод каждой найденной подстроки в терминал ноутбука. Также, был реализован алгоритм А* для нахождения максимально оптимального решения для игры в "Пятнашки".